In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt

def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return (data - mean) / std

In [3]:
housing = fetch_california_housing()

# load the data into numpy arrays
housing_features = housing.data
housing_targets = housing.target

n, f_dim = housing_features.shape

# normalize the features
housing_features= normalize(housing_features)

# add bias
housing_features = np.c_[np.ones((n, 1)), housing_features]

housing_features = housing_features[housing_targets < 5]
housing_targets = housing_targets[housing_targets < 5]

# reshape targets to TF expectation
housing_targets = np.expand_dims(housing_targets, 1)

X_train, X_test, y_train, y_test = train_test_split(housing_features,
                                                    housing_targets,
                                                    test_size=0.1)

print(housing.feature_names)
print(X_train.shape)
print(y_train.shape)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
(17683, 9)
(17683, 1)


In [4]:
LR = 0.01
EPOCHS = 5000
n_inputs = 8
n_hidden = 32
n_output = 1

In [5]:
X = tf.placeholder(tf.float32, shape=(None, f_dim+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

def dense_layer(X, n_units, name, activation=None):
    '''
    Sets up a hidden layer to be used to build a multilayer perceptron. 
    
    Initializes the weights of the neurons using a normal distribution with
    standard deviation equal to 2 / sqrt(input_dimension + number_neurons)
    
    Parameters:
    X: input data for the layer
    n_units: number of neurons to use in the layer
    name: the name of the scope to be used with this layer
    activation: the tensorflow nonlinearity to be used for each neuron
    
    Returns:
    Tensorflow graph description representing the constructed layer
    '''
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_units)
        init = tf.truncated_normal((n_inputs, n_units), stddev=stddev)
        W = tf.Variable(init, name='hidden_weights')
        b = tf.Variable(tf.zeros([n_units]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
# build the network using the hidden layer function
with tf.name_scope('dnn'):
    hidden = dense_layer(X, n_hidden, name='hidden_layer', activation=tf.nn.relu)
    y_pred = dense_layer(hidden, 1, name='output')
    
    
with tf.name_scope('loss'):
    error = y_pred - y
    loss = tf.reduce_mean(tf.square(error), name='mse')
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(LR).minimize(loss)

In [6]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

losses = []
with tf.Session() as sess:
    sess.run(init)
    
    for e in range(EPOCHS):
        if e % 100 == 0:
            print("Epoch:", e, 
                  "Current loss:", 
                  sess.run(loss, feed_dict={X: X_train, y: y_train}), 
                  "Test loss:",
                 sess.run(loss, feed_dict={X: X_test, y: y_test}))
            
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        losses.append(sess.run(loss, feed_dict={X: X_train, y: y_train}))
        
    save_path = saver.save(sess, '/tmp/mlp_regression.ckpt')

Epoch: 0 Current loss: 6.57111 Test loss: 6.60319
Epoch: 100 Current loss: 0.498092 Test loss: 0.490617
Epoch: 200 Current loss: 0.427505 Test loss: 0.428701
Epoch: 300 Current loss: 0.405509 Test loss: 0.412985
Epoch: 400 Current loss: 0.394176 Test loss: 0.405215
Epoch: 500 Current loss: 0.386787 Test loss: 0.399131
Epoch: 600 Current loss: 0.381369 Test loss: 0.39438
Epoch: 700 Current loss: 0.377212 Test loss: 0.390452
Epoch: 800 Current loss: 0.373804 Test loss: 0.387079
Epoch: 900 Current loss: 0.370866 Test loss: 0.384095
Epoch: 1000 Current loss: 0.368259 Test loss: 0.381367
Epoch: 1100 Current loss: 0.365885 Test loss: 0.378751
Epoch: 1200 Current loss: 0.363673 Test loss: 0.376333
Epoch: 1300 Current loss: 0.361594 Test loss: 0.374061
Epoch: 1400 Current loss: 0.359628 Test loss: 0.371914
Epoch: 1500 Current loss: 0.357778 Test loss: 0.369875
Epoch: 1600 Current loss: 0.356018 Test loss: 0.367926
Epoch: 1700 Current loss: 0.354331 Test loss: 0.365959
Epoch: 1800 Current loss: